In [18]:
from Tonnetz_Select import fromMidiToPCS as fmpc
from TrajectoryCalculationsWithClass import NewTrajectory as traj
from graph_creation import CreateGraph
from FirstNotePosition import PlaceFirstNote
from structural_functions import mergeDicts
import os
import pickle

In [20]:
def pickleSave(dictOfGraphs, Composer_Name) :
    completeName = "Comparison_Results/GraphObjects/"+ Composer_Name + ".p"
    with open(completeName, 'wb') as config_dictionary_file:
      # Step 3
      pickle.dump(dictOfGraphs, config_dictionary_file)

In [5]:
def GraphOfNewPiece(newPiece, directory):
    complete_name = directory + '/' + newPiece
    chordList, Tonnetz = fmpc(complete_name)
    firstPoint = PlaceFirstNote(chordList, Tonnetz)
    trajectory = traj(chordList, Tonnetz, firstPoint)
    graph = CreateGraph(trajectory)
    graph.addName(newPiece)
    return graph

In [6]:
def inputQuestions() : 
    composer = input('Do you know the Composer of this Directory ? ')
    style = input('Do you know the Style of the works in this Directory ? ')
    harmony = input('Do you know the Harmony Style of the works in this Directory ? ')
    return composer, style, harmony

In [7]:
def addingLabels(composer, style, harmony, graph):
    if composer != 'No' :
        graph.addName(composer)
    if style != 'No' :
        graph.addName(style)
    if harmony != 'No':
        graph.addName(harmony)

In [10]:
def GetWorksByDirectory(directory):
    listOfGraphs = []
    composer, style, harmony = inputQuestions()
    for file in os.listdir(directory):
        if file.endswith(".mid") or file.endswith(
                ".MID") or file.endswith(".mxl") or file.endswith(".xml"):
            try:
                print("Building Trajectory for ", file)
                graph = GraphOfNewPiece(file, directory)
                addingLabels(composer, style, harmony, graph)
                listOfGraphs.append(graph)
            except BaseException:
                print("--> Cannot build Trajectory for ", file)
    return listOfGraphs

In [28]:
jazz = GetWorksByDirectory('Midi_files/Standard_Jazz/Random')

Do you know the Composer of this Directory ? jazz
Do you know the Style of the works in this Directory ? jazz
Do you know the Harmony Style of the works in this Directory ? chromatic
Building Trajectory for  A-Getting-Sentimental-Over-You-1.mid
516
Building Trajectory for  A-Taste-Of-Honey.mid
863
Building Trajectory for  After-You-Gone.mid
1547
Building Trajectory for  All-Of-Me-2.mid
2630
Building Trajectory for  All-The-Things-You-Are.mid
2922
[0, 2, 4, 6, 8, 10] dict_items([(0, (-3, 4)), (4, (-2, 4)), (8, (-4, 4))]) (-3, 4) 7 6 3
--> Cannot build Trajectory for  All-The-Things-You-Are.mid
Building Trajectory for  All-The-Way.mid
1383
--> Cannot build Trajectory for  All-The-Way.mid
Building Trajectory for  Alone-Together.mid
874
Building Trajectory for  Antropology.mid
1680
[2, 3, 4, 7, 11] dict_items([(2, (-2, -2)), (4, (-2, -1)), (7, (-1, -1)), (11, (-3, -2))]) (-2, -2) 6 5 4
--> Cannot build Trajectory for  Antropology.mid
Building Trajectory for  Anything-Goes.mid
1172
[6, 2, 4

In [29]:
pickleSave(jazz, 'jazz')